In [ ]:
pip install pyspark

In [ ]:
#Importanto bibliotecas/funçoes

import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# resolver problema de versão python x pyspark

In [ ]:
# para iniciar a versão do pyspark
from pyspark.sql import SparkSession

#Import das funções
from pyspark.sql.functions import *

# Transforma algumas colunas em algum tipo especifico
from pyspark.sql.types import *

In [ ]:
#Criar / Iniciar sessão Spark
spark = (
    SparkSession.builder
    .master('local')
    .appName('PySpark_01')
    .getOrCreate()
)

In [ ]:
#Criar / Ler arquivo
#InferSchema ele configura os tipos de coluna, se não colocar tudo vira string
df = spark.read.csv('sample_data/wc2018-players.csv', header=True, inferSchema = True)

df.show(5)

+---------+---+----+------------------+----------+----------+--------------------+------+------+
|     Team|  #|Pos.| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
|Argentina|  3|  DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65|
|Argentina| 22|  MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65|
|Argentina| 15|  MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66|
|Argentina| 18|  DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|    69|
|Argentina| 10|  FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|   170|    72|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
only showing top 5 rows



In [ ]:
#Info as informaçoes do Schema, por para saber os tipos
df.printSchema()

root
 |-- Team: string (nullable = true)
 |-- #: integer (nullable = true)
 |-- Pos.: string (nullable = true)
 |-- FIFA Popular Name: string (nullable = true)
 |-- Birth Date: string (nullable = true)
 |-- Shirt Name: string (nullable = true)
 |-- Club: string (nullable = true)
 |-- Height: integer (nullable = true)
 |-- Weight: integer (nullable = true)



In [ ]:
#Verificar a existencia de dados nulos (No pandas)
df.toPandas().isna().sum()
#transformando em pandas. verificando oq é nulo, transformanao em pandas

Team                 0
#                    0
Pos.                 0
FIFA Popular Name    0
Birth Date           0
Shirt Name           0
Club                 0
Height               0
Weight               0
dtype: int64

In [ ]:
#Renomeando Colunas (Estavamos com um erro no nome da coluna que tinha um "."
df = df.withColumnRenamed('Pos.', 'Posicao') # (nome_antigo, nome_novo)

df.show(5)
# No Spark não substituimosa coluna, nos a sobreescrevemos


+---------+---+-------+------------------+----------+----------+--------------------+------+------+
|     Team|  #|Posicao| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|
+---------+---+-------+------------------+----------+----------+--------------------+------+------+
|Argentina|  3|     DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65|
|Argentina| 22|     MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65|
|Argentina| 15|     MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66|
|Argentina| 18|     DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|    69|
|Argentina| 10|     FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|   170|    72|
+---------+---+-------+------------------+----------+----------+--------------------+------+------+
only showing top 5 rows



In [ ]:


df = df.withColumnRenamed('Team', 'Selecao').withColumnRenamed('#', 'Numero')\
.withColumnRenamed('FIFA Popular Name', 'Nome_FIFA').withColumnRenamed('Birth Date', 'Nascimento')\
.withColumnRenamed('Shirt Name','Nome Camiseta').withColumnRenamed('Club','Time')\
.withColumnRenamed('Height', 'Altura').withColumnRenamed('Weight','Peso')

df.show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
only showing top 5 rows



In [ ]:
for coluna in df.columns:
  print(coluna, df.filter(df[coluna].isNull()).count())

Selecao 0
Numero 0
Posicao 0
Nome_FIFA 0
Nascimento 0
Nome Camiseta 0
Time 0
Altura 0
Peso 0


In [ ]:
# Selecionar Colunas
df.select('Selecao', 'Nome_FIFA').show(5)

+---------+------------------+
|  Selecao|         Nome_FIFA|
+---------+------------------+
|Argentina|TAGLIAFICO Nicolas|
|Argentina|    PAVON Cristian|
|Argentina|    LANZINI Manuel|
|Argentina|    SALVIO Eduardo|
|Argentina|      MESSI Lionel|
+---------+------------------+
only showing top 5 rows



In [ ]:
# Essa outra forma é melhor de chamar pq assim conseguimos usar funções
df.select(df['Selecao']).show(5)

# Funcoes como o Alias, por exemplo
df.select(df['Selecao'].alias('Time')).show(5)

+---------+
|  Selecao|
+---------+
|Argentina|
|Argentina|
|Argentina|
|Argentina|
|Argentina|
+---------+
only showing top 5 rows

+---------+
|     Time|
+---------+
|Argentina|
|Argentina|
|Argentina|
|Argentina|
|Argentina|
+---------+
only showing top 5 rows



Filtrar DF

In [ ]:
df.filter('Selecao = "Brazil"').show(10)

+-------+------+-------+-----------------+----------+-------------+--------------------+------+----+
|Selecao|Numero|Posicao|        Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+-------+------+-------+-----------------+----------+-------------+--------------------+------+----+
| Brazil|    18|     MF|             FRED|05.03.1993|         FRED|FC Shakhtar Donet...|   169|  64|
| Brazil|    21|     FW|           TAISON|13.01.1988|       TAISON|FC Shakhtar Donet...|   172|  64|
| Brazil|    17|     MF|      FERNANDINHO|04.05.1985|  FERNANDINHO|Manchester City F...|   179|  67|
| Brazil|    22|     DF|           FAGNER|11.06.1989|       FAGNER|SC Corinthians (BRA)|   168|  67|
| Brazil|    10|     FW|           NEYMAR|05.02.1992|    NEYMAR JR|Paris Saint-Germa...|   175|  68|
| Brazil|    11|     MF|PHILIPPE COUTINHO|12.06.1992|  P. COUTINHO|  FC Barcelona (ESP)|   172|  68|
| Brazil|     7|     FW|    DOUGLAS COSTA|14.09.1990|     D. COSTA|   Juventus FC (ITA)|   

In [ ]:
# Porem isso não vai funcionar com Colunas que tenha espaço, exemplo "Nome Camisa"
# então precisamos fazer da seguinte forma:

df.filter(col('Nome Camiseta') == "FRED").show()

+-------+------+-------+---------+----------+-------------+--------------------+------+----+
|Selecao|Numero|Posicao|Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+-------+------+-------+---------+----------+-------------+--------------------+------+----+
| Brazil|    18|     MF|     FRED|05.03.1993|         FRED|FC Shakhtar Donet...|   169|  64|
+-------+------+-------+---------+----------+-------------+--------------------+------+----+



Filtrar DF com 2 condições (AND/&)


In [ ]:
df.filter((col('selecao') == 'Argentina') & (col('Altura') > 180)).show()

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+
|Argentina|     4|     DF|  ANSALDI Cristian|20.09.1986|      ANSALDI|     Torino FC (ITA)|   181|  73|
|Argentina|     9|     FW|   HIGUAIN Gonzalo|10.12.1987|      HIGUAÍN|   Juventus FC (ITA)|   184|  75|
|Argentina|    23|     GK|CABALLERO Wilfredo|28.09.1981|    CABALLERO|    Chelsea FC (ENG)|   186|  80|
|Argentina|     2|     DF|   MERCADO Gabriel|18.03.1987|      MERCADO|    Sevilla FC (ESP)|   181|  81|
|Argentina|    17|     DF|  OTAMENDI Nicolas|12.02.1988|     OTAMENDI|Manchester City F...|   181|  81|
|Argentina|    16|     DF|       ROJO Marcos|20.03.1990|         ROJO|Manchester United...|   189|  82|
|Argentina|     6|     DF|    FAZIO Federico|17.03.1987|        

Filtrar DF com 2 condições(OR / |)

In [ ]:
df.filter((col('nome_fifa') == 'MESSI Lionel') | (col('nome_fifa') == 'SALVIO Eduardo') | (col('Altura') == 199) ).show(5)

+---------+------+-------+----------------+----------+-------------+------------------+------+----+
|  Selecao|Numero|Posicao|       Nome_FIFA|Nascimento|Nome Camiseta|              Time|Altura|Peso|
+---------+------+-------+----------------+----------+-------------+------------------+------+----+
|Argentina|    18|     DF|  SALVIO Eduardo|13.07.1990|       SALVIO|  SL Benfica (POR)|   167|  69|
|Argentina|    10|     FW|    MESSI Lionel|24.06.1987|        MESSI|FC Barcelona (ESP)|   170|  72|
|Argentina|     6|     DF|  FAZIO Federico|17.03.1987|        FAZIO|     AS Roma (ITA)|   199|  85|
|  Belgium|     1|     GK|COURTOIS Thibaut|11.05.1992|     COURTOIS|  Chelsea FC (ENG)|   199|  91|
+---------+------+-------+----------------+----------+-------------+------------------+------+----+



Criar novas colunas (usando função lit)

In [ ]:
df.withColumn('WorldCup', lit(2018)).show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+--------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|WorldCup|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+--------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|    2018|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|    2018|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|    2018|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|    2018|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|    2018|
+---------+------+-------+------------------+----------+-------------+--------------------+-----

In [ ]:
# Apesar da conta n fazer sentido, podemos ver que podemos usar o lit fazendo um calculo
df.withColumn('WorldCup', lit(col('altura') - lit(col('Peso')))).show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+--------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|WorldCup|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+--------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|     104|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|     104|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|     101|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|      98|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|      98|
+---------+------+-------+------------------+----------+-------------+--------------------+-----

Criar coluna condicional (Usando função substring)

In [ ]:
# Parametros: nome da coluna 'sub', funcao substring(nome_coluna, onde_começa, onde_termina)
df.withColumn('sub', substring('Selecao',1,3 )).show(5)


+---------+------+-------+------------------+----------+-------------+--------------------+------+----+---+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|sub|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+---+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|Arg|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|Arg|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|Arg|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|Arg|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|Arg|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+---+
only showing top 5 rows



Isto está formatado como código

In [ ]:
df.withColumn('coluna_concatenada',concat('selecao', col('nome_fifa'))).show() #concat vai criar uma na outra

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+--------------------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|  coluna_concatenada|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+--------------------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|ArgentinaTAGLIAFI...|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|ArgentinaPAVON Cr...|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|ArgentinaLANZINI ...|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|ArgentinaSALVIO E...|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|ArgentinaMESSI Li...|


In [ ]:
#outra forma é o contat_ws que pode separar as concatenaçãoes com espaço ou "-" por exemplo
df.withColumn('Separado',concat_ws(' ','Selecao', 'numero')).show(5)


+---------+------+-------+------------------+----------+-------------+--------------------+------+----+------------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso|    Separado|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+------------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65| Argentina 3|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|Argentina 22|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|Argentina 15|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|Argentina 18|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|Argentina 10|
+---------+------+-------+------------------+----------+--------

In [ ]:
#Criando a coluna Ano
df = df.withColumn('Ano', substring('Nascimento', 7, 10))
#df.withColumn('sub', substring('Selecao',1,3 )).show(5)

Alterar tipo de coluna

In [ ]:
df.printSchema()

root
 |-- Selecao: string (nullable = true)
 |-- Numero: integer (nullable = true)
 |-- Posicao: string (nullable = true)
 |-- Nome_FIFA: string (nullable = true)
 |-- Nascimento: string (nullable = true)
 |-- Nome Camiseta: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Altura: integer (nullable = true)
 |-- Peso: integer (nullable = true)
 |-- Ano: string (nullable = true)



In [ ]:
df = df.withColumn('Ano', col('Ano').cast(IntegerType()))
df.show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso| Ano|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|1992|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|1996|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|1993|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|1990|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|1987|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+
only showing top 5 

In [ ]:
df.printSchema()

root
 |-- Selecao: string (nullable = true)
 |-- Numero: integer (nullable = true)
 |-- Posicao: string (nullable = true)
 |-- Nome_FIFA: string (nullable = true)
 |-- Nascimento: string (nullable = true)
 |-- Nome Camiseta: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Altura: integer (nullable = true)
 |-- Peso: integer (nullable = true)
 |-- Ano: integer (nullable = true)



##Desafio coluna nascimento

- Colocar como datatype
- Dica : O formato deve estar em YYYY-MM-DD

In [ ]:
dia = udf(lambda data: data.split('.')[0])
df = df.withColumn('Dia',dia('nascimento')) #separar pelo . o nascimento

In [ ]:
mes = udf(lambda data: data.split('.')[1]) #funcao que criamos
df = df.withColumn('Mes',mes('nascimento'))
df.show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+---+---+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso| Ano|Dia|Mes|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+---+---+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|1992| 31| 08|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|1996| 21| 01|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|1993| 15| 02|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|1990| 13| 07|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI|  FC Barcelona (ESP)|   170|  72|1987| 24| 06|
+---------+------+-------+------------------+----------+--------

In [ ]:
df = df.withColumn('Data_nascimento', concat_ws('-', 'Ano', 'Mes', 'Dia')) # precisa necessariamente ser nessa ordem
df.withColumn('Data_nascimento', col('Data_nascimento').cast(DateType())).show()

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+---+---+---------------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome Camiseta|                Time|Altura|Peso| Ano|Dia|Mes|Data_nascimento|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+----+---+---+---------------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65|1992| 31| 08|     1992-08-31|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65|1996| 21| 01|     1996-01-21|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66|1993| 15| 02|     1993-02-15|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69|1990| 13| 07|     1990-07-13|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI